In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from evaluation.utils import entropy_batch_mixing, knn_purity
import scanpy as sc
import seaborn as sns
import scIB as scib
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_datasets/pancreas_normalized.h5ad'))
batch_key = 'study'
label_key = 'cell_type'
models = ['cvae_mse', 'cvae_nb', 'trvae_mse', 'trvae_nb']
dataset = 'pancreas'
ratios = [1, 2, 3, 4, 5]

In [ ]:
def compute_metrics(latent_adata, adata, model, dataset, rqr=None, batch_key='study', label_key='cell_type'):
    latent_adata.obsm['X_pca'] = latent_adata.X
    print(adata.shape, latent_adata.shape)
    n_batches = len(adata.obs[batch_key].unique().tolist())
    
    scores = scib.metrics.metrics(adata, latent_adata, batch_key, label_key, 
                                  nmi_=True, ari_=True, silhouette_=True, pcr_=True, graph_conn_=True, 
                                  isolated_labels_=True, hvg_score_=False)
    scores = scores.T
    scores = scores[['NMI_cluster/label', 'ARI_cluster/label', 'ASW_label', 'ASW_label/batch', 
                     'PCR_batch', 'isolated_label_F1', 'isolated_label_silhouette', 'graph_conn']]
    
    ebm = entropy_batch_mixing(latent_adata, batch_key, n_neighbors=15)
    knn = knn_purity(latent_adata, label_key, n_neighbors=15)
    
    scores['EBM'] = ebm
    scores['KNN'] = knn
    scores['method'] = model
    scores['data'] = dataset
    scores['rqr'] = rqr / n_batches if rqr is not None else None
    scores.rqr = scores.rqr.round(2)
    scores['reference_time'] = 0.0
    scores['query_time'] = 0.0
    
    return scores

## Calculating metrics for all ratios:

In [ ]:
scores = None
for model in models:
    for ratio in ratios:
        test_num = ratio
        latent_adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_results_scarches/{model}/{dataset}/test_{test_num}/full_data.h5ad'))
        latent_adata.obs[batch_key] = latent_adata.obs['batch'].values
        latent_adata.obs[label_key] = latent_adata.obs['celltype'].values  
        df = compute_metrics(latent_adata, adata, model, dataset, ratio, batch_key, label_key)
        scores = pd.concat([scores, df], axis=0) if scores is not None else df                

In [ ]:
scores

In [ ]:
scores.to_csv(os.path.expanduser(f'~/Documents/benchmarking_results_scarches/rqr_{dataset}.csv'), index=False)